In [95]:
# Opencv
import cv2 as cv

# Basic math libraries
import numpy as np 
import math

# Image displaying libraries
import pylab       
from matplotlib import pyplot as plt 
from IPython.display import HTML
from IPython.display import clear_output 
from base64 import b64encode

# Make generated images appear inline below the code
%matplotlib inline 

# Set the displayed image size
pylab.rcParams['figure.figsize'] = (10.0, 8.0) 

In [ ]:
# Download video
!wget https://raw.githubusercontent.com/Jamesrogers221194/AINT-Files/3f989bb044250abdc3c25ccc5d4c10744e649369/AINT515/Coursework01/Video1%20for%20Vision%20CW.mp4

In [118]:
### MAIN PARAMETERS ###

# Number of frames to wait after detection of droplet to prevent multiple detections
wait_frames = 10

# ROI setup
left = 60
right = 960
top = 40
bottom = 120

# Colour range to detect in the HSV color space
lower_color = (5, 50, 0)
upper_color = (42, 255, 150)

lower_color2 = (170, 50, 0)
upper_color2 = (180, 255, 150)

# X-coordinate where to count droplets + treshold
countAt = 420
counting_treshold = 4

# Accept blobs with bigger area
area_filter = 100

# Radius of droplets to accept
min_radius = 20
max_radius = 40

# Circularity of droplets to accept
circularityVal = 0.77

In [119]:
# Load input video
videoInput = cv.VideoCapture('Video1 for Vision CW.mp4')
if (videoInput.isOpened() == False): 
  print("Error opening video stream or file")

# Setup output video writer
videoOutput = cv.VideoWriter('output.avi',cv.VideoWriter_fourcc(*'MJPG'), 30, (int(videoInput.get(cv.CAP_PROP_FRAME_WIDTH)),int(videoInput.get(cv.CAP_PROP_FRAME_HEIGHT))))


# Function to count droplets
def check_x_coordinate_passing(circle_mid_x, screen_mid_x, threshold):
    delta_x = abs(circle_mid_x - screen_mid_x)

    if delta_x <= threshold:
        return True
    else:
        return False

#================================== Main Program Loop ================================

# Initialize

waitf = 0 # Start counting after waitf frames
count = 3 # Number of droplets detected

while(1):
  # Grab a frame, break the loop if there are no frames left
  ret, frame = videoInput.read()
  if(ret == False):
    break;

  # Get ROI
  roi = frame[top:bottom, left:right]
 
  ### DETECT INNER DROPLET ###

  # Convert the frame to the HSV color space
  hsv_frame = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

  # Define a mask based on the color range
  mask = cv.inRange(hsv_frame, lower_color, upper_color)

  # Create a circular brush size 5
  kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))

  # Filter image
  threshold_image = cv.erode(mask,kernel,iterations = 2)
  threshold_image = cv.dilate(threshold_image,kernel,iterations = 2)

  # Find the contours of the red droplets
  contours, hierarchy = cv.findContours(threshold_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # Find contours of interest
  for cnt in contours:
      cnt[:, :, 0] += left
      cnt[:, :, 1] += top
      area = cv.contourArea(cnt)
      if area < area_filter:
          continue

      # Calculate the center of the contour
      M = cv.moments(cnt)
      if M["m00"] != 0:
          center_x = int(M["m10"] / M["m00"])
          center_y = int(M["m01"] / M["m00"])
          center = (center_x, center_y)

          # Draw a blue contour around the blob
          cv.drawContours(frame, [cnt], 0, (255, 0, 0), 2)

          # Draw a dot in the center of the blob
          cv.circle(frame, center, 5, (255, 0, 0), -1)

          # Count droplet if center passed counting point
          if waitf == 0:
              passed_midpoint = check_x_coordinate_passing(center_x, countAt, counting_treshold)
              if passed_midpoint:
                  count += 1
                  waitf = wait_frames

  ### DETECT OUTER DROPLET ###

  # Convert the image to grayscale
  gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)

  # Apply Gaussian blur to reduce noise
  blurred = cv.GaussianBlur(gray, (5, 5), 0)

  # Create a binary image for contour detection
  _, binary = cv.threshold(blurred, 100, 255, cv.THRESH_BINARY_INV)

  # Create a circular brush size 2
  kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2))

  # Filter image
  threshold_image = cv.erode(binary,kernel,iterations = 1)
  threshold_image = cv.dilate(threshold_image,kernel,iterations = 2)

  # Find contours in the binary image
  contours, _ = cv.findContours(threshold_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # Find contours of interest
  filtered_contours = []
  for contour in contours:
      contour[:, :, 0] += left
      contour[:, :, 1] += top
      perimeter = cv.arcLength(contour, True)
      if perimeter > 0:
          area = cv.contourArea(contour)
          circularity = 4 * np.pi * (area / (perimeter * perimeter))
          if circularity > circularityVal and min_radius < np.sqrt(area / np.pi) < max_radius:
              filtered_contours.append(contour)

  # Draw contours around the circular objects
  cv.drawContours(frame, filtered_contours, -1, (0, 0, 255), 2)

  ### OUTPUT ###
  
  # Show count on video
  cv.putText(frame, f"Count: {count}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

  # Reduce the number of frames to wait
  if waitf != 0:
      waitf = waitf-1

  # Write output frame
  videoOutput.write(frame)
  
# Display output video
videoOutput.release()
print("Loading Video...")
!ffmpeg -i output.avi output.mp4 -hide_banner -loglevel error -y
clear_output()
HTML(f"""<video width=1300 controls autoplay loop><source src="{"data:video/mp4;base64," + b64encode(open('output.mp4', "rb").read()).decode()}" type="video/mp4"></video>""")